In [220]:
from SnakeBoard import SnakeBoard
from SnakeGame import SnakeGame
from NeuralNetwork import NeuralNetwork
import numpy as np
import time
import matplotlib.pyplot as plt
import pickle

In [221]:
# ---------- User defined parameters ----------
# Miscellaneous parameters
restore_weights_prev_training = 1
manual_play = 0 # Get user input (keyboard) instead of neural network auto-play
show_visuals = 0 # Show the games (1) or just play and calculated in the back-end (0)
t_between_gen = 0 # Time (secs) between generations
n_gens_2_save_weights = 1 # Num of generations elapsed to save weights in a file

# Training parameters
n_of_gens = 25 # Number of training generations
n_games_per_gen = 5 # Number of parallel games per generation
selected_games_per_gen = 2 # Selected baselines per generation to be used as references for mutations

# Mutation parameters
mrate_bias = 0.1
mrate_weights = 0.1
msize_bias = 0.1
msize_weights = 0.1

In [222]:
# ---------- Machine Learning main logic ---------- 
# Restore weights from previous training if required
if restore_weights_prev_training == 1:
    with open('./trainingHistory.bin', 'rb') as file:
        fileDataLoaded = pickle.load(file)
        file.close()

# Create 'N' games + their ANN instances
record_score, record_w_score = 0, 0
s_board = SnakeBoard(n_games_per_gen)
s_games, s_ann = [] , []
for idx in range(n_games_per_gen):
    s_games.append(SnakeGame(s_board))
    s_ann.append(NeuralNetwork())
    if restore_weights_prev_training == 1:
        s_ann[idx].set_weights_biases(fileDataLoaded["ann_weights_history"][-1][idx].weights,
                                      fileDataLoaded["ann_weights_history"][-1][idx].biases)

# Create history of the best scores and ANN weights
game_status_history, ann_weights_history = list(), list()
if restore_weights_prev_training == 1:
    game_status_history = fileDataLoaded["game_status_history"]
    ann_weights_history = fileDataLoaded["ann_weights_history"]

s_board.init_board()

# Run number of generations
for idx_gen in range(n_of_gens):
    # 1 Step all games in current generation (until all games are over)
    while True:     
        game_status = list() 
        # 1_1 Go into each game to step individually
        for idx_game, game in enumerate(s_games):
            
            #1_1_1 Get current game state and decide the next move
            state = game.get_game_state()
            if manual_play == 1:
                next_move = game.get_key()
            else:
                next_move = s_ann[idx_game].calculate(state) 
                if next_move == 0: next_move = "IDLE"
                elif next_move == 1: next_move = "T_LEFT"
                elif next_move == 2: next_move = "T_RIGHT"

            #1_1_2 Step game instance based on ANN calc. next move
            [game_over, w_score, score] = game.step_game(next_move)

            #1_1_3 Save game data in game status dictionary array
            game_status.append({"game_over":    game_over, 
                                "score":        score, 
                                "w_score":      w_score, 
                                "idx_game":     idx_game})       

        # 1_2 Update graphics of all games (visual feedback)
        if show_visuals == 1:
            s_board.clear_board()
            s_board.update_board_elements(s_games)

        # 1_3 If all game instances are over, finish current generation
        if np.min([g["game_over"] for g in game_status])==True:
            break
    
    # 2 Get best scores and ANN weights in prev. generation + save history for the last generation
    game_status.sort(key=lambda x:x["w_score"],reverse =True) # Sort from best to worst game

    game_status_history.append(list(o.copy() for o in game_status)) # Game status history
    ann_weights_history.append(list(o.copy() for o in s_ann)) # ANN weights history

    # 3 If best score in curr. gen is an all-time record, save it
    if game_status[0]["w_score"] > record_w_score:
        record_w_score = game_status[0]["w_score"]
        record_score = game_status[0]["score"]
    
    # 4 Get the best "selected_games_per_gen" games in the current generation
    # and place them in the first positions
    for i in range(selected_games_per_gen):
        s_ann[i] = s_ann[game_status[i]["idx_game"]].copy()
    
    # 5 Mutate the best ones in the subsequent positions
    for i in range(selected_games_per_gen, n_games_per_gen):
        s_ann[i] = s_ann[np.mod(i,selected_games_per_gen)].copy()
        s_ann[i].mutate(mrate_weights, msize_weights, mrate_bias, msize_bias) # random mutations

    # 6 Reset all games
    for idx_game, game in enumerate(s_games):
        game.reset_game()

    # 7 Save weights in an external file when applicable
    if np.mod(idx_gen,n_gens_2_save_weights)==0:
        with open('./trainingHistory.bin', 'wb') as file:
            fileData = {"game_status_history":    game_status_history, 
                        "ann_weights_history":    ann_weights_history}
            pickle.dump(fileData, file)
            file.close()
    
    time.sleep(t_between_gen)

    print("GEN ", idx_gen, " --- BEST SCORE: ", game_status[0]["score"] , " / ", 
            game_status[0]["w_score"] , " --- RECORD: ", record_score, " / ", record_w_score)

s_board.quit_board()

SnakeBoard instance created.
GEN  0  --- BEST SCORE:  9  /  288  --- RECORD:  9  /  288
GEN  1  --- BEST SCORE:  10  /  317  --- RECORD:  10  /  317
GEN  2  --- BEST SCORE:  9  /  282  --- RECORD:  10  /  317
GEN  3  --- BEST SCORE:  7  /  228  --- RECORD:  10  /  317
GEN  4  --- BEST SCORE:  10  /  302  --- RECORD:  10  /  317
GEN  5  --- BEST SCORE:  13  /  427  --- RECORD:  13  /  427
GEN  6  --- BEST SCORE:  12  /  384  --- RECORD:  13  /  427
GEN  7  --- BEST SCORE:  8  /  252  --- RECORD:  13  /  427
GEN  8  --- BEST SCORE:  12  /  371  --- RECORD:  13  /  427
GEN  9  --- BEST SCORE:  9  /  291  --- RECORD:  13  /  427
GEN  10  --- BEST SCORE:  9  /  293  --- RECORD:  13  /  427
GEN  11  --- BEST SCORE:  8  /  267  --- RECORD:  13  /  427
GEN  12  --- BEST SCORE:  10  /  312  --- RECORD:  13  /  427
GEN  13  --- BEST SCORE:  15  /  477  --- RECORD:  15  /  477
GEN  14  --- BEST SCORE:  9  /  282  --- RECORD:  15  /  477
GEN  15  --- BEST SCORE:  11  /  341  --- RECORD:  15  /  47